In [1]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct

In [2]:
source_file = "https://escoe-website.s3.amazonaws.com/wp-content/uploads/2018/10/05232820/BGSCSV.zip"
r = requests.get(source_file)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("downloads/")
os.listdir("downloads/")

['BGSDetails.csv', 'BGSIndices.csv', 'BGSPrices.csv', 'BGSAmounts.csv']

In [17]:
with open('downloads/BGSDetails.csv', newline='', encoding='ISO-8859-1') as f:
  reader = csv.reader(f,)
  named = False
  new_sheet = False
  for i, row in enumerate(reader):
    if new_sheet == True:
      if named != True:
        name = row[0]
        named = True
    if row[0] =="Worksheet name:":
      named = False
      new_sheet = True
    if row[0] == "Sequence":
      new_sheet = False
      headers = row
    elif (not new_sheet)&named:
      print(row)


  

['100', '4HCV64', '', '', '4.5', 'Conversion', '', '', '', '1964', '15 Jan 1959', '14 May 1959', '', '14 May 1964', '', '14 May 1964', '2', '14 May', '14 Nov', '', '', '1.470800', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'END']
['200', '2HEX6364', '', '', '2.5', 'Exchequer', '', '', '1963', '1964', '14 Nov 1954', '', '14 May 1963', '14 May 1964', '', '14 May 1964', '2', '14 May', '14 Nov', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'END']
['300', '4TY65', '', '', '4', 'Treasury', '', '', '', '1965', '4 Oct 1962', '1 Feb 1963', '', '1 Feb 1965', '', '1 Feb 1965', '2', '1 Feb', '1 Aug', '', '', '1.316600', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'END']
['400', '3SB5565', '', '', '3', 'Savings bonds', '', '', '1955', '1965', '2 Jan 1941', '', '15 Aug 1955', '15 Aug 1965', '', '15 Aug 1965', '2', '15 Feb', '15 Aug', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'END']
['500', '5HEX66', '', '', '5.

In [40]:
details.head()

,Worksheet name:,Unnamed: 1,Details,Last Updated 31Dec2024,Unnamed: 4,Unnamed: 5,New Stocks in Bold,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,END
0,Conventional stocks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END
2,NaN,New Stocks in Bold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END
3,Sequence,Inst Code,Sedol,ISIN Code,%,Stock,Suffix,Special features,First year,Last year,...,Number of calls,Call payment 1,due on,Call payment 2,due on,Call payment 3,due on,Call payment 4,due on,END
4,100,4HCV64,NaN,NaN,4.5,Conversion,NaN,NaN,NaN,1964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END


In [36]:
with open('downloads/BGSAmounts.csv', newline='', encoding='ISO-8859-1') as f:
  reader = csv.reader(f,)
  row1 = next(reader)
  row2 = next(reader)
  row3 = next(reader)
  row4 = next(reader)
  

In [37]:
row4


['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
